# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [3]:
#Your code here
import requests
import json
import pandas as pd

In [4]:
client_id = 'izsbITrHcBt4bTocKV6FTg' 
api_key = 'oAO6LwcMfDPvHWs23T8EkocM1Sq_TH-LzorNR0BXaDbsIFxT8Ybd4IGqJWkBEYkgG_mdhmtC4r3gD8m6EooLRWpyPSKeGmcfiv2BjuETY7fjUzgOINtmI20f9mzcXHYx'\

In [5]:
term = 'Pizza'
location = 'Brooklyn'


url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                
    
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text)

<Response [200]>
<class 'str'>
{"businesses": [{"id": "C8j0q4Ma_S5hBGuAI-aaww", "alias": "di-fara-pizza-brooklyn", "name": "Di Fara Pizza", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9qpe20P-vi3qFjWd4Q/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/di-fara-pizza-brooklyn?adjust_creative=izsbITrHcBt4bTocKV6FTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=izsbITrHcBt4bTocKV6FTg", "review_count": 3250, "categories": [{"alias": "pizza", "title": "Pizza"}], "rating": 4.0, "coordinates": {"latitude": 40.625093, "longitude": -73.961531}, "transactions": ["pickup", "delivery"], "price": "$$", "location": {"address1": "1424 Ave J", "address2": "", "address3": "", "city": "Brooklyn", "zip_code": "11230", "country": "US", "state": "NY", "display_address": ["1424 Ave J", "Brooklyn, NY 11230"]}, "phone": "+17182581367", "display_phone": "(718) 258-1367", "distance": 3559.746344303}, {"id": "ysqgdbSrezXgVwER2kQWKA", "alias": "julianas-pizza-brook

In [6]:
response.json()['total']

10400

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [7]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
response.json()['businesses']

[{'id': 'C8j0q4Ma_S5hBGuAI-aaww',
  'alias': 'di-fara-pizza-brooklyn',
  'name': 'Di Fara Pizza',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9qpe20P-vi3qFjWd4Q/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/di-fara-pizza-brooklyn?adjust_creative=izsbITrHcBt4bTocKV6FTg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=izsbITrHcBt4bTocKV6FTg',
  'review_count': 3250,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.625093, 'longitude': -73.961531},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '1424 Ave J',
   'address2': '',
   'address3': '',
   'city': 'Brooklyn',
   'zip_code': '11230',
   'country': 'US',
   'state': 'NY',
   'display_address': ['1424 Ave J', 'Brooklyn, NY 11230']},
  'phone': '+17182581367',
  'display_phone': '(718) 258-1367',
  'distance': 3559.746344303},
 {'id': 'ysqgdbSrezXgVwER2kQWKA',
  'alias': 'julianas-piz

In [9]:
# Your code here; use a function or loop to retrieve all the results from your original request


df = pd.DataFrame.from_dict(response.json()['businesses'])
df.head()


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,di-fara-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.625093, 'longitude': -73.961531}",(718) 258-1367,3559.746344,C8j0q4Ma_S5hBGuAI-aaww,https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9q...,False,"{'address1': '1424 Ave J', 'address2': '', 'ad...",Di Fara Pizza,+17182581367,$$,4.0,3250,"[pickup, delivery]",https://www.yelp.com/biz/di-fara-pizza-brookly...
1,julianas-pizza-brooklyn-5,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7026153030093, 'longitude': -7...",(718) 596-6700,7214.079409,ysqgdbSrezXgVwER2kQWKA,https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTx...,False,"{'address1': '19 Old Fulton St', 'address2': '...",Juliana's Pizza,+17185966700,$$,4.5,1957,[],https://www.yelp.com/biz/julianas-pizza-brookl...
2,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,6771.011722,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,416,"[pickup, delivery]",https://www.yelp.com/biz/l-industrie-pizzeria-...
3,joe-and-sals-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.6694466400113, 'longitude': -7...",(718) 484-8732,2555.936856,pQo-KWQxuH26y0IzZbQj0A,https://s3-media4.fl.yelpcdn.com/bphoto/V4YjC0...,False,"{'address1': '842 Franklin Ave', 'address2': '...",Joe & Sal's Pizza,+17184848732,$,4.5,90,"[pickup, delivery]",https://www.yelp.com/biz/joe-and-sals-pizza-br...
4,totonnos-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.578805, 'longitude': -73.983811}",(718) 372-8606,8990.516374,ZLPYcgEeRllSoeWyr7y4Ig,https://s3-media3.fl.yelpcdn.com/bphoto/DPr5z4...,False,"{'address1': '1524 Neptune Ave', 'address2': '...",Totonno's,+17183728606,$$,4.0,753,[pickup],https://www.yelp.com/biz/totonnos-brooklyn?adj...


In [10]:
lat = df['coordinates'].map(lambda x: x['latitude'])
long = df['coordinates'].map(lambda x: x['longitude'])

In [11]:
lat_df = pd.DataFrame(lat)
lat_df.rename(columns={'coordinates': 'lat'}, inplace=True)

In [12]:
long_df = pd.DataFrame(lat)
long_df.rename(columns={'coordinates': 'long'}, inplace=True)

In [13]:
coord_df = pd.concat([lat_df, long_df], axis=1)

In [14]:
df_1 = pd.concat([df, coord_df], axis =1)

In [15]:
df_1.drop(['coordinates'], axis = 1, inplace=True)

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [16]:
#Your code here
df_1.head()

,alias,categories,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,lat,long
0,di-fara-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]",(718) 258-1367,3559.746344,C8j0q4Ma_S5hBGuAI-aaww,https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9q...,False,"{'address1': '1424 Ave J', 'address2': '', 'ad...",Di Fara Pizza,+17182581367,$$,4.0,3250,"[pickup, delivery]",https://www.yelp.com/biz/di-fara-pizza-brookly...,40.625093,40.625093
1,julianas-pizza-brooklyn-5,"[{'alias': 'pizza', 'title': 'Pizza'}]",(718) 596-6700,7214.079409,ysqgdbSrezXgVwER2kQWKA,https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTx...,False,"{'address1': '19 Old Fulton St', 'address2': '...",Juliana's Pizza,+17185966700,$$,4.5,1957,[],https://www.yelp.com/biz/julianas-pizza-brookl...,40.702615,40.702615
2,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]",(718) 599-0002,6771.011722,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,416,"[pickup, delivery]",https://www.yelp.com/biz/l-industrie-pizzeria-...,40.711620,40.711620
3,joe-and-sals-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",(718) 484-8732,2555.936856,pQo-KWQxuH26y0IzZbQj0A,https://s3-media4.fl.yelpcdn.com/bphoto/V4YjC0...,False,"{'address1': '842 Franklin Ave', 'address2': '...",Joe & Sal's Pizza,+17184848732,$,4.5,90,"[pickup, delivery]",https://www.yelp.com/biz/joe-and-sals-pizza-br...,40.669447,40.669447
4,totonnos-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]",(718) 372-8606,8990.516374,ZLPYcgEeRllSoeWyr7y4Ig,https://s3-media3.fl.yelpcdn.com/bphoto/DPr5z4...,False,"{'address1': '1524 Neptune Ave', 'address2': '...",Totonno's,+17183728606,$$,4.0,753,[pickup],https://www.yelp.com/biz/totonnos-brooklyn?adj...,40.578805,40.578805


In [17]:
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
df_1.price = df_1.price.fillna(value=0)

In [20]:
price_dict = {"$": 1, "$$": 2}
df_1.price = df_1.price.map(price_dict)

In [21]:
pd.plotting.scatter_matrix(df[[]])

0     2
1     2
2     1
3     1
4     2
5     2
6     1
7     2
8     1
9     2
10    2
11    1
12    1
13    1
14    2
15    2
16    2
17    1
18    1
19    1
Name: price, dtype: int64

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [17]:
#Your code here
import folium

In [23]:
lat = df_1['lat']
long = df_1['long']

#Create a map of the area
base_map = folium.Map([lat, long])
base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!